# Connecting to the database

In [6]:
from db_tools.setup import setup
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

# Getitng a SparkSession

In [2]:
spark = setup()

23/10/26 23:29:11 WARN Utils: Your hostname, luan-Dell-G15-5520 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp0s20f3)
23/10/26 23:29:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/26 23:29:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SparkSession available as "spark"


In [3]:
def show_query(query,SparkSession = spark,n = 20):
    return SparkSession.sql(query).show()

# Set operations in SQL

<center><img width=750 src="https://n7b3p4s2.stackpathcdn.com/article/the-complete-reference-set-operations-in-ms-sql-union-all-intersect-excep/Images/e6.png"/></center>

In [13]:
film_schema = StructType([ \
    StructField("title",StringType(),True), \
    StructField("release_year",IntegerType(),True)
  ])
films = [ ('An American Pickle',2020),
          ('The Godfather',1972),
          ('Greyhound',2020)]
columns = ['title','release_year']
rdd = spark.sparkContext.parallelize(films)
spark.createDataFrame(rdd, schema = film_schema).createOrReplaceTempView('top_rated_films')

In [14]:
film_schema = StructType([ \
    StructField("title",StringType(),True), \
    StructField("release_year",IntegerType(),True)
  ])
films = [ ('The Shawshank Redemption',1994),
          ('The Godfather',1972),
          ('12 Angry Men',1957)]
columns = ['title','release_year']
rdd = spark.sparkContext.parallelize(films)
spark.createDataFrame(rdd, schema = film_schema).createOrReplaceTempView('most_popular_films')

# UNION: drop duplicates

In [15]:
query = """
SELECT * FROM top_rated_films
UNION
SELECT * FROM most_popular_films
"""
show_query(query)

+--------------------+------------+
|               title|release_year|
+--------------------+------------+
|  An American Pickle|        2020|
|       The Godfather|        1972|
|           Greyhound|        2020|
|The Shawshank Red...|        1994|
|        12 Angry Men|        1957|
+--------------------+------------+



# UNION ALL: does NOT drop duplicates

In [19]:
query = """ SELECT title,release_year,COUNT(title) AS frequency
FROM(
SELECT * FROM top_rated_films
UNION ALL
SELECT * FROM most_popular_films)
GROUP BY title,release_year
ORDER BY frequency DESC
"""
show_query(query)

+--------------------+------------+---------+
|               title|release_year|frequency|
+--------------------+------------+---------+
|       The Godfather|        1972|        2|
|  An American Pickle|        2020|        1|
|           Greyhound|        2020|        1|
|The Shawshank Red...|        1994|        1|
|        12 Angry Men|        1957|        1|
+--------------------+------------+---------+



# INTERSECT

In [22]:
query = """ 
SELECT * FROM top_rated_films
INTERSECT
SELECT * FROM most_popular_films
"""
show_query(query)

+-------------+------------+
|        title|release_year|
+-------------+------------+
|The Godfather|        1972|
+-------------+------------+



# EXCEPT

In [23]:
query = """ 
SELECT * FROM top_rated_films
EXCEPT
SELECT * FROM most_popular_films
"""
show_query(query)

+------------------+------------+
|             title|release_year|
+------------------+------------+
|         Greyhound|        2020|
|An American Pickle|        2020|
+------------------+------------+



In [24]:
query = """ 
SELECT * FROM most_popular_films
EXCEPT
SELECT * FROM top_rated_films
"""
show_query(query)

+--------------------+------------+
|               title|release_year|
+--------------------+------------+
|The Shawshank Red...|        1994|
|        12 Angry Men|        1957|
+--------------------+------------+

